In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import igraph as ig

from networkx.algorithms import community

In [3]:
df = pd.read_csv('docs/static/data/graph.csv')
df = df[['MUNIC_MOV','MUNIC_RES','HOSPITALIZACOES']]
df = df[df['MUNIC_MOV'] != df['MUNIC_RES']]
df.head()

,MUNIC_MOV,MUNIC_RES,HOSPITALIZACOES
1,Aquidabã - Sergipe,Aracaju - Sergipe,2
2,Aquidabã - Sergipe,Canhoba - Sergipe,11
3,Aquidabã - Sergipe,Capela - Sergipe,23
4,Aquidabã - Sergipe,Gracho Cardoso - Sergipe,2
5,Aquidabã - Sergipe,Malhada dos Bois - Sergipe,9


In [4]:
G = nx.from_pandas_edgelist(df, 
                           source='MUNIC_MOV', 
                           target='MUNIC_RES', 
                           edge_attr='HOSPITALIZACOES',
                           create_using=nx.DiGraph())

In [5]:
# Convert to igraph format
edges = [(df.iloc[i]['MUNIC_MOV'], df.iloc[i]['MUNIC_RES']) for i in range(len(df))]
weights = df['HOSPITALIZACOES'].tolist()
g_ig = ig.Graph.TupleList(edges, directed=True)
g_ig.es['weight'] = weights

# Apply Leiden algorithm
communities = g_ig.community_infomap(edge_weights='weight', vertex_weights=None)

print(f"Number of communities found: {len(communities)}")

Number of communities found: 874


In [6]:
# Get community membership for each vertex
membership = communities.membership

# Create a DataFrame with municipality-community mapping
community_df = pd.DataFrame({
    'municipality': [g_ig.vs[i]['name'] for i in range(len(g_ig.vs))],
    'community_id': membership
})

In [7]:
community_df.to_csv('docs/static/data/communities.csv')

In [8]:
df = df.join(community_df.set_index('municipality'), on = 'MUNIC_RES')
df.rename(columns = { 'community_id': 'COMMUNITY_RES' }, inplace = True)
df = df.join(community_df.set_index('municipality'), on = 'MUNIC_MOV')
df.rename(columns = { 'community_id': 'COMMUNITY_MOV' }, inplace = True)


In [9]:
community_graph = df.groupby(['COMMUNITY_RES', 'COMMUNITY_MOV']).agg({'HOSPITALIZACOES':'sum'}).reset_index()
community_graph

,COMMUNITY_RES,COMMUNITY_MOV,HOSPITALIZACOES
0,0,0,105
1,0,1,54
2,0,7,15
3,0,8,1
4,0,11,1
...,...,...,...
7631,870,108,23
7632,870,870,22
7633,871,108,13
7634,872,108,7


In [12]:
groups = community_graph.groupby('COMMUNITY_RES')

percentages = { res: group[group['COMMUNITY_MOV'] == res]['HOSPITALIZACOES'].sum() / group['HOSPITALIZACOES'].sum() for res, group in groups}

In [15]:
zeros = [community for community in percentages if percentages[community] == 0.0]

In [ ]:
df[df['COMMUNITY_MOV'].isin(zeros)]